# DSI-ML-workshop: Deep Learning and Transfer Learning

Attribution: Kolhatkar, Varada (2024) DSCI572 

## Imports
<hr>

In [ ]:
import numpy as np
import pandas as pd
from collections import OrderedDict
import torch
from torch import nn, optim
from torchvision import datasets, transforms, utils, models
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from PIL import Image

plt.rcParams.update({'axes.grid': False})

<br><br>

## Getting Started with Kaggle Kernels
<hr>

We are going to run this notebook on the cloud using [Kaggle](https://www.kaggle.com). Kaggle offers 30 hours of free GPU usage per week which should be much more than enough for this lab. To get started, follow these steps:

1. Go to https://www.kaggle.com/kernels

2. Make an account if you don't have one, and verify your phone number (to get access to GPUs)
3. Select `+ New Notebook`
<img src="img/create_notebook.png" alt="drawing" width="500"/>
4. Go to `File -> Import Notebook`
5. Upload this notebook
6. On the right-hand side of your Kaggle notebook, make sure:
  
  - `Internet` is enabled.
  
  - In the `Accelerator` dropdown, choose `GPU` when you're ready to use it (you can turn it on/off as you need it).
<img src="img/session_options.png" alt="drawing" width="300"/>
7. In Kaggle Notebook, running the follow cell should print out `"Using device: cuda"` which means a GPU is available:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device.type}")

Once you've done all your work on Kaggle, you can download the notebook from Kaggle. That way any work you did on Kaggle won't be lost. 

## Exercise 1: Transfer Learning
<hr>

In this exercise you're going to practice transfer learning. We're going to develop a model that can detect the following 6 cat breeds in this Kaggle [dataset](https://www.kaggle.com/solothok/cat-breed):

0. American Short hair
1. Bengal
2. Maine Soon
3. Ragdoll
4. Scottish Fold
5. Sphinx

In order to use this dataset 

1. Click `+ Add data` at the top right of the notebook.

<img src="img/add_data1.png" alt="drawing" width="300"/>

2. Search for 'cat-breed-mardhik'. Several datasets will appear. Look for and 'Add' the dataset with the specific thumbnail below, which has a size of 93 MB.

<img src="img/add_data2.png" alt="drawing" width="300"/>
<img src="img/cat-breed.png" alt="drawing" width="500"/>

### Feature Extractor

**Your tasks:**

In this exercise, you will build a CNN model to classify images of cats based on their breeds! 

However, training a CNN model from scratch requires a lot of computation resources. So, you will leverage a pre-trained model customized with your own layer(s) on top, to build a CNN classifier that can identify various cat breeds!

First, run the follow cell for preparation of the data and model training setup.

In [ ]:
# Set up data
TRAIN_DIR = "/kaggle/input/cat-breed/TRAIN/"
VALID_DIR = "/kaggle/input/cat-breed/TEST/"

def data_loader(DIR):
    IMAGE_SIZE = 200
    BATCH_SIZE = 32
    
    data_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    dataset = datasets.ImageFolder(root=DIR, transform=data_transforms)
    loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    return loader

train_loader = data_loader(TRAIN_DIR)
valid_loader = data_loader(VALID_DIR)

# Set up trainer
def trainer(model, train_loader, valid_loader, epochs=20, verbose=True):
    """Simple training wrapper for PyTorch network."""

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
    
    train_loss, valid_loss, train_accuracy, valid_accuracy = [], [], [], []
    for epoch in range(epochs):  # for each epoch
        train_batch_loss = 0
        train_batch_acc = 0
        valid_batch_loss = 0
        valid_batch_acc = 0
        
        # Training
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            _, y_hat_labels = torch.softmax(y_hat, dim=1).topk(1, dim=1)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_batch_loss += loss.item()
            train_batch_acc += (y_hat_labels.squeeze() == y).type(torch.float32).mean().item()
        train_loss.append(train_batch_loss / len(train_loader))
        train_accuracy.append(train_batch_acc / len(train_loader))
        
        # Validation
        model.eval()
        with torch.no_grad():
            for X, y in valid_loader:
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                _, y_hat_labels = torch.softmax(y_hat, dim=1).topk(1, dim=1)
                loss = criterion(y_hat, y)
                valid_batch_loss += loss.item()
                valid_batch_acc += (y_hat_labels.squeeze() == y).type(torch.float32).mean().item()
        valid_loss.append(valid_batch_loss / len(valid_loader))
        valid_accuracy.append(valid_batch_acc / len(valid_loader))
        
        # Print progress
        if verbose:
            print(f"Epoch {epoch + 1}:",
                  f"Train Loss: {train_loss[-1]:.3f}",
                  f"Train Accuracy: {train_accuracy[-1]:.2f}",
                  f"Valid Loss: {valid_loss[-1]:.3f}.",
                  f"Valid Accuracy: {valid_accuracy[-1]:.2f}")
    
    results = {"train_loss": train_loss,
               "train_accuracy": train_accuracy,
               "valid_loss": valid_loss,
               "valid_accuracy": valid_accuracy}
    return results

def plot_samples(data_loader, model=None):
    sample_batch = next(iter(data_loader))
    plt.figure(figsize=(20, 16)); 
    plt.axis("off"); 
    plt.title("Sample Images")
    plt.imshow(np.transpose(utils.make_grid(sample_batch[0], padding=1, normalize=True),(1, 2, 0)));
    actual_labels = sample_batch[1].numpy()
    print(f"Actual Labels: {actual_labels}")
    if model:
        _, y_hat_labels = torch.softmax(cnn_model(sample_batch[0].to(device)), dim=1).topk(1, dim=1)
        predicted_labels = y_hat_labels.squeeze().cpu().numpy()
        print(f"Predicted labels: {predicted_labels}")
        print(f"Accuracy: {np.mean(actual_labels == predicted_labels)}")
        
    return

>If you want to take a look at the images after making a `train_loader`, try this code:

In [ ]:
# Plot samples
plot_samples(train_loader)

Then, you will use any pre-trained model you wish (`DenseNet` for this case) and start training your model for 10 epochs by running the follow cells.

In [ ]:
# Download model and freeze params
cnn_model = models.densenet121(pretrained=True)
for param in cnn_model.parameters():
    param.requires_grad = False

# Customize final classification layers
new_layers = nn.Sequential(
    nn.Linear(1024, 50),
    nn.ReLU(),
    nn.Linear(50, 6)
)
cnn_model.classifier = new_layers

In [ ]:
# Time to train
epochs = 10
results = trainer(cnn_model, train_loader, valid_loader, epochs)

> If you want to take a look at the images in validation set and compare the actual and the predicted labels, try this code:

In [ ]:
# Plot samples
plot_samples(valid_loader, cnn_model)

**Food for Thought**: 

- How is the performance of the model?
- What do you think about the change in the model performance if you change the value of `epoch` and train the model again?

_Type your answer here, replacing this text._

### Fine Tuning

**Your tasks:**

In this exercise, you will fine-tune your pre-trained model by updating more layers during training.

You can fine-tune as many layers as you like: the whole model, or particular layers. Experiment with both modes of fine-tuning, and find which works better.

You will start fine-tuning the last 2 layers of the model by running the follow cells.

In [ ]:
# Download model and freeze params
cnn_model = models.densenet121(pretrained=True)
for param in cnn_model.parameters():
    param.requires_grad = False

# Customize final classification layers
new_layers = nn.Sequential(
    nn.Linear(1024, 50),
    nn.ReLU(),
    nn.Linear(50, 6)
)
cnn_model.classifier = new_layers

print(f"The number of layers in the model is {len(cnn_model.features)}")

In [ ]:
# Customize number of fine-tuning layers
num_fine_tuning_layer = 2
if num_fine_tuning_layer > 0:
    for layer in cnn_model.features[-num_fine_tuning_layer:]:
        for param in layer.parameters():
            param.requires_grad = True

In [ ]:
# Time to train
epochs = 10
results = trainer(cnn_model, train_loader, valid_loader, epochs)

> If you want to take a look at the images in validation set and compare the actual and the predicted labels, try this code:

In [ ]:
# Plot samples
plot_samples(valid_loader, cnn_model)

**Food for Thought**: 

- How is the performance of the model? Comment on the performance of this model compared to the "feature extractor" model.
- What do you think about the change in the model performance if you change the value of `num_fine_tuning_layer` and train the model again? Do you notice any change in running time?

_Type your answer here, replacing this text._

### Your Free Time

**Your tasks**:

You will add any labelled image dataset you like, use your own settings by changing the items with **{_ ... _}** and train your own model!

Feel free to share your thoughts with your teammates and workshop team.

In [ ]:
TRAIN_DIR = "{_TRAIN_FILE_PATH_}"
VALID_DIR = "{_VALID_FILE_PATH_}"
# Example
# TRAIN_DIR = "/kaggle/input/cat-breed/TRAIN/"
# VALID_DIR = "/kaggle/input/cat-breed/TEST/"

train_loader = data_loader(TRAIN_DIR)
valid_loader = data_loader(VALID_DIR)

In [ ]:
# Plot samples
plot_samples(train_loader)

In [ ]:
def model_init(num_label_classes, num_fine_tuning_layer):
    # Download model and freeze params
    model = models.densenet121(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    
    # Customize final classification layers
    new_layers = nn.Sequential(
        nn.Linear(1024, 50),
        nn.ReLU(),
        nn.Linear(50, num_label_classes)
    )
    model.classifier = new_layers
    
    # Customize number of fine-tuning layers
    if num_fine_tuning_layer > 0:
        for layer in model.features[-num_fine_tuning_layer:]:
            for param in layer.parameters():
                param.requires_grad = True

    return model

In [ ]:
num_label_classes = "{_number_of_label_classes_in_the_dataset_}"
num_fine_tuning_layer = "{_number_of_fine_tuning_layer_}"
epochs = "{_epochs_}"
# Example
# num_label_classes = 6
# num_fine_tuning_layer = 1
# epochs = 10

In [ ]:
# Time to train
cnn_model = model_init(num_label_classes, num_fine_tuning_layer)
results = trainer(cnn_model, train_loader, valid_loader, epochs)

In [ ]:
# Plot samples
plot_samples(valid_loader, cnn_model)

<!-- END QUESTION -->

<br><br>

![](img/eva-congrats.png)

<br><br>